<a href="https://colab.research.google.com/github/abhiram4/Weld-length-prediction/blob/main/MaskR_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies
!pip install torch torchvision
!pip install pycocotools
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [ ]:
import torch
import torchvision
import cv2
import numpy as np
from matplotlib import pyplot as plt
from roboflow import Roboflow

In [ ]:
# Download the dataset from Roboflow
rf = Roboflow(api_key="HSvdWoVcQ0H4zdWTiCTN")
project = rf.workspace().project("weld-dimension-prediction")
dataset = project.version(1).download("coco")  # Replace with your version number

loading Roboflow workspace...
loading Roboflow project...
Exporting format coco in progress : 85.0%
Version export complete for coco format



Extracting Dataset Version Zip to Weld-dimension-prediction-1 in coco:: 100%|██████████| 39/39 [00:00<00:00, 4213.10it/s]


In [ ]:
from torchvision.models.detection import maskrcnn_resnet50_fpn

# Load pre-trained Mask R-CNN model
model = maskrcnn_resnet50_fpn(pretrained=True)
model.eval()  # Set the model to evaluation mode
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)


The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth
100%|██████████| 170M/170M [00:01<00:00, 124MB/s]


MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [ ]:
# Function to preprocess images
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_tensor = torchvision.transforms.functional.to_tensor(image).to(device)
    return image, image_tensor

# Function to calculate pixel-to-cm ratio
def calculate_pixel_to_cm_ratio(detections, known_lengths):
    for box, label in zip(detections['boxes'], detections['labels']):
        if label == 1:  # 'ruler'
            pixel_length = box[2] - box[0]
            return known_lengths['ruler'] / pixel_length
        elif label == 2:  # 'pen'
            pixel_length = box[2] - box[0]
            return known_lengths['pen'] / pixel_length
    raise ValueError("No reference object found!")

# Function to calculate weld length
def calculate_weld_length(detections, pixel_to_cm_ratio):
    weld_length = 0
    for box, label in zip(detections['boxes'], detections['labels']):
        if label == 3:  # 'welds'
            pixel_length = box[2] - box[0]
            weld_length += pixel_length * pixel_to_cm_ratio
    return weld_length


In [ ]:
# Known lengths of objects in cm
known_lengths = {'ruler': 31.5, 'pen': 15.0}

# Load an image and make predictions
image_path = "/content/WIN_20240902_19_35_13_Pro.jpg"  # Replace with an image path
image, image_tensor = preprocess_image(image_path)

# Perform inference
with torch.no_grad():
    predictions = model([image_tensor])

# Process detections
detections = predictions[0]
print("Detections:", detections)
print("Boxes:", detections['boxes'])
print("Labels:", detections['labels'])
# Filter detections by confidence score
confidence_threshold = 0.5  # Adjust as needed
filtered_boxes = []
filtered_labels = []

for i, score in enumerate(detections['scores']):
    if score > confidence_threshold:
        filtered_boxes.append(detections['boxes'][i])
        filtered_labels.append(detections['labels'][i])

detections['boxes'] = filtered_boxes
detections['labels'] = filtered_labels

print("Filtered Detections:", detections)


Detections: {'boxes': tensor([[   0.0000,  666.1436, 1758.9926, 1032.8038]]), 'labels': tensor([15]), 'scores': tensor([0.0789]), 'masks': tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])}
Boxes: tensor([[   0.0000,  666.1436, 1758.9926, 1032.8038]])
Labels: tensor([15])
Filtered Detections: {'boxes': [], 'labels': [], 'scores': tensor([0.0789]), 'masks': tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])}


In [ ]:
# Known lengths of objects in cm
known_lengths = {'ruler': 31.5, 'pen': 15.0}

# Load an image and make predictions
image_path = "/content/WIN_20240902_19_35_13_Pro.jpg"  # Replace with an image path
image, image_tensor = preprocess_image(image_path)

# Perform inference
with torch.no_grad():
    predictions = model([image_tensor])

# Process detections
detections = predictions[0]
pixel_to_cm_ratio = calculate_pixel_to_cm_ratio(detections, known_lengths)
weld_length_cm = calculate_weld_length(detections, pixel_to_cm_ratio)

# Display results
print(f"Predicted weld length: {weld_length_cm:.2f} cm")

# Visualize detections
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.title(f"Weld Length: {weld_length_cm:.2f} cm")
plt.axis("off")
plt.show()
